In [ ]:
# pip install bert-extractive-summarizer

In [ ]:
pip install pymystem3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import string
import numpy as np
import pandas as pd 
import re
from keras.preprocessing.text import Tokenizer 
from nltk import download
download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
from attention import AttentionLayer
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# reviews = pd.read_csv("news_summary_more.csv")
!pwd
import os
os.chdir("drive/MyDrive/Colab Notebooks")

# !ls
reviews = pd.read_csv("dataset.csv")
# reviews = pd.read_csv("news_dataset.csv")

print(reviews.shape)
reviews.head()

/content
(99999, 4)


,Unnamed: 0,Title,Text,url
0,0,"Чего ждут казахстанцы от партии Nur Otan - исследование\n 20 ноября 2019, 17:29\n","\nИнститут общественной политики озвучил итоги проведенного в ноябре социологического исследования, сообщает пресс-служба партии.\nКак отметила директор института Юлия Кучинская, по итогам опроса ...",https://tengrinews.kz/kazakhstan_news/chego-jdut-kazahstantsyi-ot-partii-nur-otan-issledovanie-384534/
1,1,"Отца ""шымкентского мажора"" осудили на 11 лет колонии\n 22 августа 2018, 15:50\n","\nВ уголовном суде Туркестанской области завершилось дело в отношении Ерехана Сулейменова, известного в криминальном мире как ""Ероха Туркестанский"", а также получившего широкую ""популярность"" как ...",https://tengrinews.kz/kazakhstan_news/ottsa-shyimkentskogo-majora-osudili-na-11-let-kolonii-352067/
2,2,"Еще 5 рейсов отменили в аэропортах Астаны и Алматы\n 10 февраля 2016, 17:08\n","\nВ аэропортах Астаны и Алматы по состоянию на 16.00 10 февраля отменены пять рейсов, передает корреспондент Tengrinews.kz.\nТак, в южной воздушной гавани отменен рейс авиакомпании Air Astana в Ас...",https://tengrinews.kz/kazakhstan_news/esche-5-reysov-otmenili-v-aeroportah-astanyi-i-almatyi-288793/
3,3,"57 радикалов осудили в Казахстане с начала 2016 года - КНБ\n 05 сентября 2016, 16:59\n","\nС начала 2016 года в Казахстане осудили 57 радикалов, передает корреспондент Tengrinews.kz со ссылкой на КНБ РК.\nПо данным ведомства, с начала 2016 года органами национальной безопасности за со...",https://tengrinews.kz/kazakhstan_news/57-radikalov-osudili-v-kazahstane-s-nachala-2016-goda-knb-301610/
4,4,"Как казахстанцы отдохнут в декабре\n 13 ноября 2020, 15:22\n","\nВ декабре этого года у казахстанцев будет 20 рабочих дней и 11 выходных, передает корреспондент Tengrinews.kz.\nПродолжение\n\n\n1.\n\nФейерверка 16 декабря в Нур-Султане не будет\n\n\n\n\n\n\n\...",https://tengrinews.kz/kazakhstan_news/kak-kazahstantsyi-otdohnut-v-dekabre-420062/


In [ ]:
# Check for any nulls values
reviews.isnull().sum()

Unnamed: 0    0
Title         8
Text          8
url           0
dtype: int64

In [7]:
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

EMAIL_REGEX = re.compile(
    r"(?:^|(?<=[^\w@.)]))([\w+-](\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(\.([a-z]{2,})){1,3}(?:$|(?=\b))",
    flags=re.IGNORECASE | re.UNICODE,)

table = str.maketrans('', '', string.punctuation)
# mystem = Mystem() 

def clean_text_rus(text, remove_stopwords = True):
    text = text.lower()
    # too slow
    # text = mystem.lemmatize(text.lower())
    text = EMAIL_REGEX.sub(' ',text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r"'s\b","", text)
    text = re.sub(r'&amp;', '', text) 
    if remove_stopwords:
        text = text.split()
        text = [w.translate(table) for w in text]
        stops = set(stopwords.words("russian"))
        # stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = [word for word in text if word.isalpha()]
        text = [word.lower() for word in text]
        # text = [word.split() for word in text]
        text = [word for word in text if len(word) > 1]

    return text    

In [9]:
table_title = reviews['Title'][:4000]
# table_title = reviews['headlines'][:4000]
table_title.tail()

3995                    Школьники перейдут на удаленку в Павлодарской области\n                04 декабря 2020, 18:15\n            
3996              Кайрат Лама Шариф разъяснил понятие "светское государство"\n                29 сентября 2011, 11:30\n            
3997                                     В Шымкенте могут закрыть семь рынков\n                03 октября 2012, 20:42\n            
3998                   В Норвегии состоялось вручение Нобелевской премии мира\n                10 декабря 2011, 19:52\n            
3999    В Казахстане завершилась одна забастовка нефтяников и началась другая\n                02 февраля 2021, 04:44\n            
Name: Title, dtype: object

In [10]:
table_texts = reviews['Text'][:4000]
# table_texts = reviews['text'][:4000]
table_texts.tail()

3995    \nШкольники перейдут на дистанционку в городах Павлодарской области. Из-за роста заболеваемости COVID-19 на дистанционное обучение перейдут школы Павлодара, Экибастуза и Аксу, передает корреспонде...
3996    \n                            На понятии "светское государство" отдельно остановился председатель Агентства РК по делам религий во время обсуждения законопроекта "О религиозной деятельности и рели...
3997    \n                            В Шымкенте под угрозой закрытия оказались семь рынков, сообщает телеканал "Отырар". Прокурорская проверка выявила огромный список нарушений. Здесь и отсутствие автопа...
3998    \n                            В городской ратуше норвежской столицы Осло в субботу прошла церемония награждения Нобелевской премией мира за 2011 год, передает "Интерфакс-Казахстан" со ссылкой на м...
3999    \nБастовавшие несколько дней рабочие компаний "КМК Мунай" и "АМК Мунай" в Темирском районе Актюбинской области приступили к работе, добившись повышения заработн

In [11]:

cleaned_news = []
for texts in table_texts:
  cleaned_text3 = clean_text_rus(texts)
  cleaned_news.append(cleaned_text3)

print(cleaned_news[:5])

[['институт', 'общественной', 'политики', 'озвучил', 'итоги', 'проведенного', 'ноябре', 'социологического', 'исследования', 'сообщает', 'пресс', 'служба', 'партии', 'как', 'отметила', 'директор', 'института', 'юлия', 'кучинская', 'по', 'итогам', 'опроса', 'был', 'составлен', 'топ', 'вопросов', 'из', 'регионов', 'которые', 'партия', 'nur', 'otan', 'должна', 'взять', 'на', 'контроль', 'первую', 'очередь', 'это', 'повышение', 'уровня', 'заработной', 'платы', 'противодействие', 'коррупции', 'решение', 'проблем', 'сферах', 'образования', 'здравоохранения', 'также', 'борьба', 'социальным', 'неравенством', 'ноябре', 'нами', 'был', 'проведен', 'очередной', 'замер', 'мнений', 'по', 'общественно', 'политической', 'ситуации', 'отношению', 'конкретно', 'партии', 'nur', 'otan', 'опросе', 'участвовали', 'респонденты', 'представляющие', 'все', 'регионы', 'страны', 'опрос', 'был', 'проведен', 'форме', 'face', 'facе', 'акцент', 'был', 'сделан', 'на', 'изучении', 'динамики', 'рейтинга', 'партии', 'оценк

In [12]:
cleaned_titles = []
for headlines in table_title:
  cleaned_text3 = clean_text_rus(headlines)
  cleaned_titles.append(cleaned_text3)

print(cleaned_titles[:5])

[['чего', 'ждут', 'казахстанцы', 'от', 'партии', 'nur', 'otan', 'исследование', 'ноября'], ['отца', 'шымкентского', 'мажора', 'осудили', 'на', 'лет', 'колонии', 'августа'], ['еще', 'рейсов', 'отменили', 'аэропортах', 'астаны', 'алматы', 'февраля'], ['радикалов', 'осудили', 'казахстане', 'начала', 'года', 'кнб', 'сентября'], ['как', 'казахстанцы', 'отдохнут', 'декабре', 'ноября']]


In [19]:
max_headlines_len=10
max_text_len=400

cleaned_text = cleaned_news
cleaned_headlines = cleaned_titles

short_text=[]
short_headlines=[]

for i in range(len(cleaned_text)):
    if( len(cleaned_headlines[i]) <= max_headlines_len and len(cleaned_text[i])<=max_text_len ):
        short_text.append(' '.join(cleaned_text[i]))
        short_headlines.append(' '.join(cleaned_headlines[i]))


df=pd.DataFrame({'text':short_text,'headlines':short_headlines})

df['headlines'] = df['headlines'].apply(lambda x : 'sostok '+ x + ' eostok')


from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['headlines']),test_size=0.1,random_state=0,shuffle=True)
df.head()

len(cleaned_headlines) 4000
len(cleaned_text) 4000


,text,headlines
0,институт общественной политики озвучил итоги проведенного ноябре социологического исследования сообщает пресс служба партии как отметила директор института юлия кучинская по итогам опроса был сост...,sostok чего ждут казахстанцы от партии nur otan исследование ноября eostok
1,уголовном суде туркестанской области завершилось дело отношении ерехана сулейменова известного криминальном мире как ероха туркестанский также получившего широкую популярность как отец шымкентског...,sostok отца шымкентского мажора осудили на лет колонии августа eostok
2,аэропортах астаны алматы по состоянию на февраля отменены пять рейсов передает корреспондент tengrinews kz так южной воздушной гавани отменен рейс авиакомпании air astana астану он планировался на...,sostok еще рейсов отменили аэропортах астаны алматы февраля eostok
3,начала года казахстане осудили радикалов передает корреспондент tengrinews kz со ссылкой на кнб рк по данным ведомства начала года органами национальной безопасности за совершение преступлений тер...,sostok радикалов осудили казахстане начала года кнб сентября eostok
4,декабре этого года казахстанцев будет рабочих дней выходных передает корреспондент tengrinews kz продолжение фейерверка декабря нур султане не будет так помимо стандартных субботы воскресенья каза...,sostok как казахстанцы отдохнут декабре ноября eostok


In [20]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

x_voc

% of rare words in vocabulary: 72.61630860250395
Total Coverage of rare words: 12.43888996894781


18330

In [21]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

thresh=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

print("y_tokenizer.word_counts = ",y_tokenizer.word_counts)    
print("y_tokenizer.word_counts.items() = ",y_tokenizer.word_counts.items()) 

print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_headlines_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_headlines_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

y_tokenizer.word_counts['sostok'],len(y_tr)   

print("y_voc = ",y_voc)
print("y_tokenizer.word_counts['sostok'] = ",y_tokenizer.word_counts['sostok'])
print("len(y_tr)  = ",len(y_tr) )

y_tokenizer.word_counts =  OrderedDict([('sostok', 3000), ('министр', 43), ('обороны', 17), ('проверил', 3), ('алматинский', 1), ('гвардейский', 1), ('жетыгенский', 1), ('гарнизоны', 1), ('июня', 242), ('eostok', 3000), ('жанаозене', 11), ('планируют', 8), ('построить', 5), ('теплицы', 1), ('марта', 277), ('карагандинцев', 1), ('просят', 12), ('сохранять', 2), ('спокойствие', 2), ('августа', 248), ('потасовка', 1), ('между', 14), ('журналистами', 1), ('иностранных', 7), ('сми', 27), ('произошла', 3), ('на', 462), ('встрече', 3), ('астане', 92), ('января', 211), ('мвд', 26), ('установлен', 2), ('провокатор', 1), ('кордайского', 1), ('конфликта', 1), ('февраля', 287), ('более', 28), ('человек', 36), ('погибли', 14), ('из', 158), ('за', 191), ('сильных', 1), ('дождей', 1), ('индии', 6), ('ким', 1), ('чен', 1), ('ын', 1), ('стал', 25), ('генсеком', 1), ('трудовой', 1), ('партии', 5), ('северной', 2), ('кореи', 1), ('таможне', 1), ('кордай', 1), ('назначен', 33), ('новый', 33), ('начальник'

In [22]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)


y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)


ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)


In [23]:
from keras import backend as K 
K.clear_session()

latent_dim = 200
embedding_dim=110

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 400)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 110)     2016300     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 400, 200), ( 248800      embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [25]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

history=model.fit([x_tr,y_tr[:,:-1]],
                  y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:],
                  epochs=1,
                  callbacks=[es],
                  batch_size=128, 
                  validation_data=([x_val,y_val[:,:-1]],
                                   y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

24/24 [==============================] - 503s 20s/step - loss: 2.7530 - val_loss: 2.3897


In [26]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.index_word
target_word_index = y_tokenizer.word_index

print("y_tokenizer = ",y_tokenizer)
print("reverse_target_word_index = ",reverse_target_word_index)
print("reverse_target_word_index = ",reverse_source_word_index)
print("reverse_target_word_index = ",target_word_index)


# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_headlines_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

y_tokenizer =  <keras_preprocessing.text.Tokenizer object at 0x7f33fbd46110>
reverse_target_word_index =  {1: 'sostok', 2: 'eostok', 3: 'на', 4: 'апреля', 5: 'алматы', 6: 'февраля', 7: 'марта', 8: 'октября', 9: 'декабря', 10: 'казахстане', 11: 'ноября', 12: 'августа', 13: 'мая', 14: 'июня', 15: 'сентября', 16: 'июля', 17: 'января', 18: 'за', 19: 'казахстана', 20: 'из', 21: 'области', 22: 'назарбаев', 23: 'по', 24: 'не', 25: 'от', 26: 'астане', 27: 'тенге', 28: 'для', 29: 'нур', 30: 'токаев', 31: 'казахстан', 32: 'коронавируса', 33: 'казахстанцев', 34: 'рк', 35: 'султане', 36: 'астаны', 37: 'коронавирусом', 38: 'как', 39: 'министр', 40: 'рассказал', 41: 'об', 42: 'года', 43: 'могут', 44: 'тысяч', 45: 'рассказали', 46: 'человек', 47: 'видео', 48: 'будут', 49: 'после', 50: 'алматинской', 51: 'назначен', 52: 'новый', 53: 'лет', 54: 'до', 55: 'миллионов', 56: 'глава', 57: 'что', 58: 'более', 59: 'аким', 60: 'сми', 61: 'году', 62: 'сша', 63: 'мвд', 64: 'под', 65: 'президент', 66: 'стал', 67:

In [27]:
for i in range(0,100):
    print("News text:",seq2text(x_tr[i]))
    print("Original summary:",seq2text(y_tr[i]))
    # print("Original summary:",seq2summary(y_tr[i]))
    # print("predicted = ",seq2summary(x_tr[i]) )
    # print("predicted = ",x_tr[i].reshape(1,max_text_len))
    # print("")
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    # print("Predicted summary:",decode_sequence(x_tr[i]))
    print("\n")

News text: глава военного ведомства генерал полковник сакен жасузаков совершил рабочую поездку части алматинского ходе работы глава ведомства проверил места проведения конкурсов условия проживания команд участниц армейских международных игр маршрутов также порядок преодоления препятствий отметив важность проведения крупнейших соревнований профессиональной мастерства военнослужащих генерал полковник поставил ряд конкретных задач руководству вооруженных сил по дальнейшему выполнению работ проведению арми следует отметить что текущем году на территории казахстана пройдут три конкурса частности мастера огня рубеж соревнования расчетов беспилотных летательных аппаратов ходе инспекции министр обороны осмотрел объекты войсковой части также рамках рабочей поездки южный регион генерал полковник жасузаков проверил работу военно патриотического лагеря жас сарбаз на территории учебно центра спортивного комитета центрального спортивного клуба армии мо рк который начал функционировать впервые для де